In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

C:\Users\Cédric Symons\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<h1>Question 9</h1>

In [2]:
T = 6  # Num periods
rep = 1  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

Min:       $\sum_{t=1}^{T} (f_t y_t + c_t x_t + h_t s_t + b_t r_t)$

Subject to: 

$s_{t-1} + x_t - r_{t-1} = d_t + s_t - r_t$, for all $t = 1,...,T$

$x_t \leq D_{1T} y_t$, for all $t = 1,...,T$

$s_0 = s_T = r_0 =r_T = 0$

$x_t \geq 0$, $s_t \geq 0$, $r_t \geq 0$, $y \in \{0,1\}$

In [3]:
def SI_ULSPwB():
    start_time = time.time()
    m = gp.Model("Uncapacitated_Lot_Sizing_Problem_with_Backlogging")

    # decision variables
    x = m.addVars(T+1, vtype=GRB.INTEGER, name="x")  # x
    s = m.addVars(T+1, vtype=GRB.INTEGER, name="s")  # s
    r = m.addVars(T+1, vtype=GRB.INTEGER, name="r")  # r
    y = m.addVars(T+1, vtype=GRB.BINARY, name="y")  # y

    # objective function
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t], b[t]], [y[t], x[t], s[t], r[t]])

    m.setObjective(obj, GRB.MINIMIZE)

    # constraints
    for t in H:
        m.addConstr(s[t - 1] + x[t] - r[t - 1] == d[t] + s[t] - r[t], f"inventory_balance_{t}")
#         m.addConstr(s[t - 1] + x[t]  == d[t] + s[t] , f"inventory_balance_{t}")
        m.addConstr(x[t] <= D[0] * y[t], f"production_constraint_{t}")
                    
    m.addConstr(s[0] == 0, "initial_inventory")
    m.addConstr(r[0] == 0, "initial_backlog")
    m.addConstr(y[0] == 0)
    m.addConstr(s[T] == 0, "final_inventory")
    m.addConstr(r[T] == 0, "final_backlog")
    # Solve the model
    m.optimize()

    # Print solution
    if m.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print("Period\t\tDemand\tSetup\tProduction\tHolding\tBacklogging\tProduction\tBacklogging\tHolding\tSetup")
        for t in H:
            print(f"Period_{t}\t{d[t]}\t{f[t]}\t{c[t]}\t\t{h[t]}\t{b[t]}\t\t{x[t].X}\t\t{r[t].X}\t\t{s[t].X}\t\t{y[t].X}")
        print(f"Total cost: {m.objVal}")
    else:
        print("No solution found.")
    end_time = time.time()
    total_time = end_time - start_time
    print('Total Time:',total_time)
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))
    r_values = np.zeros(len(H_0))
    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X
        r_values[t] = r[t].X
    cost = m.objVal
    return cost, x_values, y_values, s_values, r_values

In [4]:
total_cost, x, y, s, r = SI_ULSPwB()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-07
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 28 columns and 47 nonzeros
Model fingerprint: 0xb37afd83
Variable types: 0 continuous, 28 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Found heuristic solution: objective 48840.000000
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolved: 12 rows, 22 columns, 38 nonzeros
Variable types: 0 continuous, 22 integer (6 binary)

Root relaxation: objective 2.706359e+04, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

In [5]:
total_cost = []
for i in [6, 12, 24, 52, 104]:
    T = i  # Num periods
    for j in [1, 2]:
        rep = j  # Tab number

        tabs = {'6-1': '6-periods (1)',
            '6-2': '6-periods (2)',
            '12-1': '12-periods (1)',
            '12-2': '12-periods (2)',
            '24-1': '24-periods (1)',
            '24-2': '24-periods (2)',
            '52-1': '52-periods (1)',
            '52-2': '52-periods (2)',
            '104-1': '104-periods (1)',
            '104-2': '104-periods (2)'}

        xls = pd.ExcelFile(r'ULSP-instancesR.xlsx')

        df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

        # Add zero row and adjust index to align with indicies prompt
        df.loc[-1] = [0]*6
        df.index = df.index + 1
        df = df.sort_index()

        # Constants
        d = df['Demand Forecast']
        f = df['Setup Cost']
        c = df['Production cost']
        h = df['Holding cost']
        b = df['Backlogging cost']

        D = [d[i:].sum() for i in range(len(d))]

        # Set
        H = range(1,T+1)
        H_0 = range(T+1)
        
        cost,_,_,_,_ = SI_ULSPwB()
        
        total_cost.append(cost)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 28 columns and 47 nonzeros
Model fingerprint: 0xb37afd83
Variable types: 0 continuous, 28 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Found heuristic solution: objective 48840.000000
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolved: 12 rows, 22 columns, 38 nonzeros
Variable types: 0 continuous, 22 integer (6 binary)

Root relaxation: objective 2.706359e+04, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     


CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 53 rows, 100 columns and 173 nonzeros
Model fingerprint: 0xfb1bc668
Variable types: 0 continuous, 100 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 9e+02]
Found heuristic solution: objective 149001.00000
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolved: 48 rows, 94 columns, 164 nonzeros
Variable types: 0 continuous, 94 integer (24 binary)

Root relaxation: objective 7.263034e+04, 54 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 72630.3367    0   19 149001.000 72630.3367  51.3%     -    0s
H    0     0

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 126750.936    0   42 322204.000 126750.936  60.7%     -    0s
H    0     0                    320473.00000 126750.936  60.4%     -    0s
H    0     0                    315199.00000 126750.936  59.8%     -    0s
H    0     0                    307774.00000 126750.936  58.8%     -    0s
H    0     0                    296544.00000 126750.936  57.3%     -    0s
H    0     0                    285482.00000 178261.555  37.6%     -    0s
     0     0 178261.555    0   69 285482.000 178261.555  37.6%     -    0s
H    0     0                    279835.00000 179788.899  35.8%     -    0s
H    0     0                    234431.00000 179788.899  23.3%     -    0s
H    0     0                    234089.00000 179788.899  23.2%     -    0s
     0     0 189085.494    0   77 234089.000 189085.494  19.2%     -    0s
H    0     0                    234073.00000 192405.616  17.8%     -    0s
H    0     0          

  Implied bound: 9
  MIR: 48
  Flow cover: 38
  Flow path: 16
  Relax-and-lift: 1

Explored 1 nodes (297 simplex iterations) in 0.11 seconds (0.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 201802 202074 203621 ... 262379

Optimal solution found (tolerance 1.00e-04)
Best objective 2.018020000000e+05, best bound 2.018020000000e+05, gap 0.0000%
Optimal solution found!
Period		Demand	Setup	Production	Holding	Backlogging	Production	Backlogging	Holding	Setup
Period_1	614	6105	6		1	4		-0.0		614.0		-0.0		-0.0
Period_2	0	6311	5		1	5		-0.0		614.0		-0.0		-0.0
Period_3	612	608	5		3	3		1497.0		-0.0		271.0		1.0
Period_4	271	946	8		8	4		-0.0		-0.0		-0.0		-0.0
Period_5	376	3501	7		1	4		622.0		-0.0		246.0		1.0
Period_6	61	2970	9		9	3		-0.0		-0.0		185.0		-0.0
Period_7	185	8936	5		2	3		-0.0		-0.0		-0.0		-0.0
Period_8	0	5518	9		7	6		-0.0		-0.0		-0.0		-0.0
Period_9	479	8645	9		5	3		-0.0		479.0		-0.0		-0.0
Period_10	0	6052	7		8	5		615.0		-0.0		136.0		1.0
Period_11	136	59


CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 213 rows, 420 columns and 733 nonzeros
Model fingerprint: 0x55f8983d
Variable types: 0 continuous, 420 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [1e+00, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 1e+03]
Found heuristic solution: objective 1698330.0000
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolved: 208 rows, 414 columns, 724 nonzeros
Variable types: 0 continuous, 414 integer (104 binary)

Root relaxation: objective 3.682058e+05, 275 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 368205.772    0   82 1698330.00 368205.772  78.3%     -    0s
H    

<h1>Question 10</h1>

One of the patterns that can be seen is the model always does the demand of a period in one period; so in other words it won't do half of a period in one period and half in another. This seems normal as it will choose the lowest combination of setup, production/ holding/ backlogging cost.

There will never be holding and backlogging costs in the same period as this would be inprofitable.

Some interesting properties:

$s_{t-1} > 0$ implies $x_t = 0$


$x_t > 0$ implies $r_t = 0$

$s_{t-1} > 0$ implies $r_t = 0$

In [6]:

def SI_ULSPwB_DP():
      
    T = len(d)-1  # Number of periods
    def D_tu(t, u):
        if t == u:
            return d[t]
        return sum(d[t:u+1])
    a_alpha = [0] * (T+1)  
    b_beta = [[0] * k for k in range(T+2)]
    gamma_star =[0] * (T+1)
    beta_star = [0] * (T+1)
    gamma_star = T
    beta_star = T
    
    for alpha, beta in zip(range(T,-1,-1), range(T+1,0,-1)):
        for e in range(beta):
            b_betas = []
            for gamma in range(beta, gamma_star+1):
                b_betas.append(c[beta]*sum([d[k] for k in range(beta-e, gamma+1)]) + (sum([h[k]*D_tu(k+1, gamma) for k in range(beta, gamma)])+a_alpha[gamma]))
            if b_betas:
                b_beta[beta][e] = f[beta] + min(b_betas)
    #             gamma_star = T #max([beta + index for index, element in enumerate(b_betas) if element == min(b_betas)])
        
        a_alphas = []
        for Beta in range(alpha+1, beta_star+1):
            a_alphas.append(sum([b[k]*D_tu(alpha+1, k) for k in range(alpha, Beta)])+b_beta[Beta][Beta-alpha-1])

        if a_alphas:
            a_alpha[alpha] = min(a_alphas)
        
#             beta_star = T #max([alpha+1 + index for index, element in enumerate(a_alphas) if element == min(a_alphas)])
    
    return a_alpha[0]


In [7]:
SI_ULSPwB_DP()

664029

In [8]:
total_cost_DP = []
for i in [6, 12, 24, 52]:
    T = i  # Num periods
    for j in [1, 2]:
        rep = j  # Tab number

        tabs = {'6-1': '6-periods (1)',
            '6-2': '6-periods (2)',
            '12-1': '12-periods (1)',
            '12-2': '12-periods (2)',
            '24-1': '24-periods (1)',
            '24-2': '24-periods (2)',
            '52-1': '52-periods (1)',
            '52-2': '52-periods (2)',
            '104-1': '104-periods (1)',
            '104-2': '104-periods (2)'}

        xls = pd.ExcelFile(r'ULSP-instancesR.xlsx')

        df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

        # Add zero row and adjust index to align with indicies prompt
        df.loc[-1] = [0]*6
        df.index = df.index + 1
        df = df.sort_index()

        # Constants
        d = df['Demand Forecast']
        f = df['Setup Cost']
        c = df['Production cost']
        h = df['Holding cost']
        b = df['Backlogging cost']

        D = [d[i:].sum() for i in range(len(d))]

        # Set
        H = range(1,T+1)
        H_0 = range(T+1)
        start_time = time.time()
        cost = SI_ULSPwB_DP()
        end_time = time.time()
        total_time = end_time - start_time
        print(total_time, [f'{T}-{rep}'])
        total_cost_DP.append(cost)

0.004998922348022461 ['6-1']
0.003999233245849609 ['6-2']
0.041999101638793945 ['12-1']
0.039998769760131836 ['12-2']
0.5941598415374756 ['24-1']
0.6216416358947754 ['24-2']
11.959247589111328 ['52-1']
11.979140043258667 ['52-2']


In [9]:
print(total_cost, total_cost_DP)

[29911.0, 101686.0, 53662.0, 44598.0, 78316.0, 119187.0, 203337.0, 201802.0, 331305.0, 664029.0] [29911, 101686, 53662, 44598, 78316, 119187, 203337, 201802]


<h1>Question 11</h1>

Min:       $\sum_{t=1}^{T} ()$

Subject to: 

$w_t + $

In [27]:
T = 6  # Num periods
rep = 1  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

In [28]:
#TODO: in progress
def SI_ULSPwB_SPR():

    
    def D_ut(u,t):
        return sum(d[u:t+1])
        
    # Create a new model
    m = gp.Model("Uncapacitated_Lot_Sizing_Problem_with_Backlogging_Shortest_Path_Reformulation")
    
    
    # Decision variables
    w = m.addVars(T+1, vtype=GRB.BINARY, name="w")  # Binary variable indicating whether the demand for period t is produced in period t
    phi = m.addVars(T+1, T+1, vtype=GRB.BINARY, name="phi")  # Binary variable = 1 if production in u includes the future demand precisely up to period t ≥ u, and 0 otherwise;
    psi = m.addVars(T+1, T+1, vtype=GRB.BINARY, name="psi")  # Binary variable = 1 if production in u includes backlogged demand precisely from period t ≤ u, and 0 otherwise;
    y = m.addVars(T+1, vtype=GRB.BINARY, name="y")  # y

    # Objective function
    m.setObjective(
        gp.quicksum(f[t] * w[t] +  
                    c[t] * w[t] * d[t] +
                    gp.quicksum(phi[u, t] * (D_ut(u+1, t) * c[u] + gp.quicksum(D_ut(i,t)*h[i] for i in range(u+1, t+1))) for u in range(0, t)) +
                    gp.quicksum(psi[u, t] * (D_ut(t, u-1) * c[u] + gp.quicksum(D_ut(t,i)*b[i] for i in range(t, u))) for u in range(t, T+1))
                    for t in H),
        GRB.MINIMIZE
    )
#     obj = gp.LinExpr()

#     # Demand satisfied by production in current period
#     for t in H:
#         obj.addTerms(d[t]*c[t]+f[t], w[t])

#     for u in H:

#         # Demand satisfied by production in later period
#         for t in H: # range(u+1, T+1):
#             obj.addTerms(d[t]*(c[u]+sum([b[i] for i in range(t,u)])), psi[u,t])

#         # Demand satisfied by production in earlier period
#         for t in H: # range(1,u):
#             obj.addTerms(d[t]*(c[u]+sum([h[j] for j in range(u,t)])), phi[u,t])


#     m.setObjective(obj, GRB.MINIMIZE)

    # Constraints
    m.addConstr(gp.quicksum(phi[u, T] for u in range(1,T+1)) == 1) # ending constraint
    m.addConstr(gp.quicksum(psi[u, 1] for u in range(1,T+1)) == 1) # starting constraint
    
    m.addConstrs(w[t] == y[t] for t in range(1,T+1))
    
    for u in H:
        
        # assignments
        m.addConstrs(phi[u,t] == 0 for t in H[:u-1])
        m.addConstrs(psi[u,t] == 0 for t in H[u+1:])
        
    # when producing in u previous patterns showed it would also produce it's own demand:
    #constraint between node i' and i''
    m.addConstrs(gp.quicksum(phi[u, t] for t in H) - w[u] == 0 for u in H) 
    #constraint between node i and i'
    m.addConstrs(gp.quicksum(psi[u, t] for t in H) - w[u] == 0 for u in H)
    #constraint between node i'' and i+1
    m.addConstrs(gp.quicksum(phi[u, t] for u in H) - gp.quicksum(psi[u, t+1] for u in H[1:]) == 0 for t in range(1,T))
                
    m.optimize()

    # Print solution
#     if m.status == GRB.OPTIMAL:
#         print("Optimal solution found!")
#         for v in m.getVars():
#             print(f"{v.VarName}: {v.X}")
#         print(f"Total cost: {m.objVal}")
#     else:
#         print("No solution found.")

    cost = m.objVal
    return cost

In [29]:
SI_ULSPwB_SPR()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 112 columns and 188 nonzeros
Model fingerprint: 0x90307443
Variable types: 0 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 203190.00000
Presolve removed 34 rows and 65 columns
Presolve time: 0.00s
Presolved: 16 rows, 47 columns, 94 nonzeros
Variable types: 0 continuous, 47 integer (47 binary)

Root relaxation: objective 2.991100e+04, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*  

29911.0

In [30]:
total_cost_SPR = []
for i in [6, 12, 24, 52, 104]:
    T = i  # Num periods
    for j in [1, 2]:
        rep = j  # Tab number

        tabs = {'6-1': '6-periods (1)',
            '6-2': '6-periods (2)',
            '12-1': '12-periods (1)',
            '12-2': '12-periods (2)',
            '24-1': '24-periods (1)',
            '24-2': '24-periods (2)',
            '52-1': '52-periods (1)',
            '52-2': '52-periods (2)',
            '104-1': '104-periods (1)',
            '104-2': '104-periods (2)'}

        xls = pd.ExcelFile(r'ULSP-instancesR.xlsx')

        df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

        # Add zero row and adjust index to align with indicies prompt
        df.loc[-1] = [0]*6
        df.index = df.index + 1
        df = df.sort_index()

        # Constants
        d = df['Demand Forecast']
        f = df['Setup Cost']
        c = df['Production cost']
        h = df['Holding cost']
        b = df['Backlogging cost']

        D = [d[i:].sum() for i in range(len(d))]

        # Set
        H = range(1,T+1)
        H_0 = range(T+1)
        
        cost = SI_ULSPwB_SPR()
        
        total_cost_SPR.append(cost)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 112 columns and 188 nonzeros
Model fingerprint: 0x90307443
Variable types: 0 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 203190.00000
Presolve removed 34 rows and 65 columns
Presolve time: 0.00s
Presolved: 16 rows, 47 columns, 94 nonzeros
Variable types: 0 continuous, 47 integer (47 binary)

Root relaxation: objective 2.991100e+04, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*  


CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2810 rows, 5724 columns and 13574 nonzeros
Model fingerprint: 0xf3b47fa8
Variable types: 0 continuous, 5724 integer (5724 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2656 rows and 2870 columns
Presolve time: 0.02s
Presolved: 154 rows, 2854 columns, 5708 nonzeros
Variable types: 0 continuous, 2854 integer (2854 binary)
Found heuristic solution: objective 2429731.0000
Found heuristic solution: objective 1915549.0000

Root relaxation: objective 1.975230e+05, 150 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   

In [31]:
print(total_cost_SPR, total_cost)

[29911.0, 88534.0, 48823.0, 44740.0, 78386.0, 122193.0, 197523.0, 194813.0, 328735.0, 679395.0] [29911.0, 101686.0, 53662.0, 44598.0, 78316.0, 119187.0, 203337.0, 201802.0, 331305.0, 664029.0]
